In [1]:
import torch
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)


In [2]:
from torch import nn, optim
import torch.nn.functional as F

# Define the network architecture
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(x.shape[0], -1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)

        return x

# Create the network, define the criterion and optimizer
model = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)


In [3]:
epochs = 5

for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        # Clear the gradients, do this because gradients are accumulated
        optimizer.zero_grad()

        # Forward pass, get our log-probabilities
        log_ps = model(images)
        # Calculate the loss with the logps and the labels
        loss = criterion(log_ps, labels)

        # Calling .backward() on loss calculates the gradients for us
        loss.backward()

        # Take an update step and few the new weights
        optimizer.step()

        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")


Training loss: 2.2528244536568613


Training loss: 1.7510574662100786


Training loss: 0.8799086618525133


Training loss: 0.5947929923214129


Training loss: 0.48675338495006437


In [4]:
# Download and load the test data

testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)


In [5]:
# Implement a function for the validation pass

def validation(model, testloader, criterion):
    test_loss = 0
    accuracy = 0

    for images, labels in testloader:

        log_ps = model(images)
        test_loss += criterion(log_ps, labels)

        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

    return test_loss, accuracy

model.eval()

with torch.no_grad():
    test_loss, accuracy = validation(model, testloader, criterion)

print(f"Test loss: {test_loss/len(testloader)}")
print(f"Test accuracy: {accuracy/len(testloader)}")


Test loss: 0.43762731552124023
Test accuracy: 0.8725119233131409
